In [1]:
# #Import modules

# import pandas as pd
# # from IPython.display import display, HTML
# # from numpy import log10, nan, inf, random, clip, arange

# # path = './../export_france/data/type1/DP1610_MAASTRICHT1_1997_2013'

# # display(HTML("<style>.container { width:100% !important; }</style>"))

# import matplotlib.pyplot as plt
# %matplotlib inline

# import dask.dataframe as dd
# from dask.diagnostics import ProgressBar

# from numpy import arange, log10

# from functions import chunk, agg, finalize
# tunique = dd.Aggregation('tunique', chunk, agg,finalize)
# first = dd.Aggregation('first', chunk, agg,finalize)

### Count buyer-seller links

In [ ]:
# dd.read_csv('2014-*.csv')

# drive_path = './../export_france/data/type1/DP1610_MAASTRICHT1_1997_2013'
drive_path = './../../../../media/miglesia/Elements/export_france/data/type1/DP1610_MAASTRICHT1_1997_2013'

n_bins = 20
df_list = n_bins * [ 0 ]

for dataset_i in range(n_bins):
    df_list[dataset_i] = dd.read_csv(drive_path+'/samplings/YMxpb_size20'+str(dataset_i).zfill(2)+'.csv')#,
#                                     usecols = ['YEAR','ID','VAT','VART_sum'])

data = dd.concat(df_list)
data = data.groupby(['YEAR','ID','VAT'])['VART_sum'].sum().reset_index()

In [ ]:
with ProgressBar():
    out = data.compute()
out.to_csv('buyer_seller_links.csv', index = False)

data = dd.from_pandas(out, npartitions = 100)

In [142]:
links = pd.read_csv('./buyer_seller_links.csv')
links = links.loc[links.VART_sum > 1000]
links_count = links.groupby('YEAR').agg({'VART_sum': 'count', 'ID': 'nunique', 'VAT': 'nunique'})
links_count = links_count.rename(columns = {'VART_sum': 'pairs', 'ID': 'FR sellers', 'VAT': 'foreign buyers'})
links_pct_change = 100*links_count.pct_change().round(4).fillna('')

In [143]:
links_summary = pd.concat([links_count,links_pct_change], axis = 1)
links_summary.columns = pd.MultiIndex.from_product([['count', 'yearly pct change'], links_count.columns])

In [144]:
links_summary

count                               yearly D%               \
      # pairs # FR sellers # foreign buyers   # pairs # FR sellers   
YEAR                                                                 
1997   810865        48482           349416                          
1998   853034        53380           356449       5.2         10.1   
1999   879117        54320           367328      3.06         1.76   
2000   900988        54481           381432      2.49          0.3   
2001   887826        50259           384110     -1.46        -7.75   
2002   899477        48530           390763      1.31        -3.44   
2003   899695        48273           397693      0.02        -0.53   
2004   955009        48682           426010      6.15         0.85   
2005   974115        48417           429763         2        -0.54   
2006   980558        46912           434310      0.66        -3.11   
2007  1002149        45967           450843       2.2        -2.01   
2008   985853        45141           447878     -1.63         -1.8   
2009   930321        43192           430183     -5.63        -4.32   
2010   946037        42463           437200      1.69        -1.69   
2011   834748        26690           406142    -11.76       -37.15   
2012   830036        24846           404678     -0.56        -6.91   
2013   829460        24768           403480     -0.07        -0.31   

                       
     # foreign buyers  
YEAR                   
1997                   
1998             2.01  
1999             3.05  
2000             3.84  
2001              0.7  
2002             1.73  
2003             1.77  
2004             7.12  
2005             0.88  
2006             1.06  
2007             3.81  
2008            -0.66  
2009            -3.95  
2010             1.63  
2011             -7.1  
2012            -0.36  
2013             -0.3

In [157]:
# buyer_sizes = links.groupby(['VAT', 'YEAR'])[['VART_sum']].sum()
# buyer_sizes.to_csv('./buyer_sizes.csv')
seller_sizes = links.groupby(['ID', 'YEAR'])[['VART_sum']].sum()
seller_sizes.to_csv('./seller_sizes.csv')